In [5]:
import pandas as pd
import pymysql

# Establish initial connection to MySQL without specifying a database
myconnection = pymysql.connect(host='localhost', user='root', passwd='Santhi@18')

# Create cursor object
cursor = myconnection.cursor()

# Create the database if it does not exist
cursor.execute("CREATE DATABASE IF NOT EXISTS redbus_data_scraping")

# Close the initial connection
cursor.close()
myconnection.close()

# Establish connection to the newly created database
myconnection = pymysql.connect(host='localhost', user='root', passwd='Santhi@18', database='redbus_data_scraping')

# Create cursor object
cursor = myconnection.cursor()

# Drop table if it exists (optional)
table_name = "bus_routes"
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

# Read the CSV file
df = pd.read_csv("redbus_final_data.csv")

# Ensure correct column names and drop unnecessary columns
df = df.rename(columns=lambda x: x.strip())  # Remove leading/trailing whitespaces

# Rename columns to match expected names, including 'state_name'
df.rename(columns={
    'Bus Name': 'busname',
    'Bus Type': 'bustype',
    'Departure Time': 'departing_time',
    'Arrival Time': 'reaching_time',
    'Seat_Availability': 'seats_available',  # Ensure this matches
    'Bus Link': 'route_link',
    'Bus Route': 'route_name',
    'Duration': 'duration',
    'Fare': 'price',
    'Rating': 'star_rating',
    'Seat Availability': 'seats_available',  # Fix this column name
    'State Name': 'state_name'  # Add state_name column
}, inplace=True)

# Print the columns to debug
print("Columns in DataFrame:", df.columns)

# Add a 'state_name' column if it doesn't exist, or ensure it's filled
if 'state_name' not in df.columns:
    df['state_name'] = 'Unknown'  # or any default value you prefer

# Format the DataFrame columns
try:
    if 'departing_time' in df.columns:
        df['departing_time'] = pd.to_datetime(df['departing_time'], format='%H:%M', errors='coerce').dt.time
    if 'reaching_time' in df.columns:
        df['reaching_time'] = pd.to_datetime(df['reaching_time'], format='%H:%M', errors='coerce').dt.time
except Exception as e:
    print(f"Error parsing date columns: {e}")

# Clean 'seats_available' column and handle NaN values
if 'seats_available' in df.columns:
    df['seats_available'] = df['seats_available'].astype(str).str.extract(r'(\d+)').fillna(0).astype(int)
else:
    print("Warning: 'seats_available' column not found")

# Ensure 'star_rating' and 'price' columns are properly formatted
if 'star_rating' in df.columns:
    df['star_rating'] = pd.to_numeric(df['star_rating'], errors='coerce').fillna(0)  # Convert to float and fill NaNs with 0
if 'price' in df.columns:
    df['price'] = pd.to_numeric(df['price'], errors='coerce').fillna(0)  # Convert to float and fill NaNs with 0

# Ensure 'busname' column is present and treated as string
if 'busname' in df.columns:
    df['busname'] = df['busname'].astype(str).fillna('Unknown')
else:
    print("Warning: 'busname' column not found")

# Map pandas dtypes to MySQL dtypes, specifying DECIMAL for star_rating
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'INT',
    'int32': 'INT',
    'float64': 'FLOAT',
    'datetime64[ns]': 'DATETIME',
    'datetime.time': 'TIME'
}

# Specify the order of columns, including 'state_name' at the beginning
ordered_columns = ['state_name', 'route_name', 'route_link', 'busname', 'bustype', 'departing_time', 'duration', 'reaching_time', 'star_rating', 'price', 'seats_available']

# Generate the formatted string for creating the table with escaped column names and types
column_definitions = ["`id` INT AUTO_INCREMENT PRIMARY KEY"]  # Add primary key column
for col in ordered_columns:
    if col == 'star_rating':
        column_definitions.append(f"`{col}` DECIMAL(10, 2)")
    else:
        column_definitions.append(f"`{col}` {dtype_mapping.get(str(df[col].dtype), 'TEXT')}")
table_definition = ", ".join(column_definitions)

# Create table
create_table_query = f"CREATE TABLE {table_name} ({table_definition})"
print("Create Table Query:", create_table_query)  # Print the query to debug
cursor.execute(create_table_query)

# Insert data into the table
insert_query = f"INSERT INTO {table_name} ({', '.join([f'`{col}`' for col in ordered_columns])}) VALUES ({', '.join(['%s'] * len(ordered_columns))})"

# Execute the insert queries
for row in df[ordered_columns].itertuples(index=False, name=None):
    cursor.execute(insert_query, row)

# Commit the changes
myconnection.commit()

# Close the cursor and connection
cursor.close()
myconnection.close()

Columns in DataFrame: Index(['busname', 'bustype', 'departing_time', 'reaching_time', 'duration',
       'price', 'star_rating', 'seats_available', 'route_link', 'route_name',
       'state_name'],
      dtype='object')
Create Table Query: CREATE TABLE bus_routes (`id` INT AUTO_INCREMENT PRIMARY KEY, `state_name` TEXT, `route_name` TEXT, `route_link` TEXT, `busname` TEXT, `bustype` TEXT, `departing_time` TEXT, `duration` TEXT, `reaching_time` TEXT, `star_rating` DECIMAL(10, 2), `price` FLOAT, `seats_available` INT)
